In [26]:
objects = [
    {
    "Obj1_url":"data_test/pairwise/breaking_bad_statue_pair1/breaking_bad_cat_statue_piece1.ply",
    "Obj2_url":"data_test/pairwise/breaking_bad_statue_pair1/breaking_bad_cat_statue_piece2.ply",
    "small_object":30000,"large_object":30000,
    #to1,to2,to3 -> always fixed for the object creation (i fixed it for almost of them)
    "to1":100 ,"to2":100, "to3":100,
    #tb1,tb2,tb3 -> border paramters depend on the nature of the object
    "tb1":0.1 ,"tb2":1, "tb3":0.1,
    #dilation_size , thre -> size of the dilation and threshold applied depend on each object
    "dilation_size":0.02,"thre":0.96},
    {
    "Obj1_url":"data_test/pairwise/breaking_bad_cat_artifact_pair2/breaking_bad_cat_artifact_piece2.ply",
    "Obj2_url":"data_test/pairwise/breaking_bad_cat_artifact_pair2/breaking_bad_cat_artifact_piece3.ply",
    "small_object":30000,"large_object":30000,
    "to1":100 ,"to2":100, "to3":100,
    "tb1":0.1 ,"tb2":1, "tb3":0.1,
    "dilation_size":0.02,"thre":0.96},
    {
    "Obj1_url":"data_test/pairwise/breaking_bad_cat_artifact_pair1/breaking_bad_cat_artifact_piece1.ply",
    "Obj2_url":"data_test/pairwise/breaking_bad_cat_artifact_pair1/breaking_bad_cat_artifact_piece3.ply",
    "small_object":30000,"large_object":30000,
    "to1":100 ,"to2":100, "to3":100,
    "tb1":0.1 ,"tb2":1, "tb3":0.1,
    "dilation_size":0.02,"thre":0.96}
]

In [27]:
def get_winner_pair(test):
    import scipy
    from copy import copy
    i = 0
    winner_index = -1
    winner_value = 10000000
    arr = []
    for o1,o2,R_T in test.result_transformation_arr:
    #     print(o1,o2)
        obj1 = test.Obj1_array[o1]
        obj2 = test.Obj2_array[o2]
        obj2_tmp = copy(obj2)
        obj2_tmp.pcd = copy(obj2.pcd)
        obj2_tmp.pcd.transform(R_T)
    #     print(scipy.spatial.distance.directed_hausdorff(list(obj2_tmp.pcd.points), list(obj1.pcd.points), seed=0))
    #     print(scipy.spatial.distance.directed_hausdorff(list(obj1.pcd.points),list(obj2_tmp.pcd.points) , seed=0))
        chamfer_value = chamfer_distance(list(obj2_tmp.pcd.points),list(obj1.pcd.points))
        arr.append({"index":i,"o1":o1,"o2":o2,"chamfer":chamfer_value})
        if winner_value>chamfer_value:
            winner_index = i
            winner_value = chamfer_value
        i+=1
    chamfer_df = pd.DataFrame(arr)
    return chamfer_df.sort_values('chamfer')
import numpy as np
from sklearn.neighbors import NearestNeighbors


def chamfer_distance(x, y, metric='l2', direction='bi'):
    """Chamfer distance between two point clouds
    Parameters
    ----------
    x: numpy array [n_points_x, n_dims]
        first point cloud
    y: numpy array [n_points_y, n_dims]
        second point cloud
    metric: string or callable, default ‘l2’
        metric to use for distance computation. Any metric from scikit-learn or scipy.spatial.distance can be used.
    direction: str
        direction of Chamfer distance.
            'y_to_x':  computes average minimal distance from every point in y to x
            'x_to_y':  computes average minimal distance from every point in x to y
            'bi': compute both
    Returns
    -------
    chamfer_dist: float
        computed bidirectional Chamfer distance:
            sum_{x_i \in x}{\min_{y_j \in y}{||x_i-y_j||**2}} + sum_{y_j \in y}{\min_{x_i \in x}{||x_i-y_j||**2}}
    """
    
    if direction=='y_to_x':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        chamfer_dist = np.mean(min_y_to_x)
    elif direction=='x_to_y':
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_x_to_y)
    elif direction=='bi':
        x_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(x)
        min_y_to_x = x_nn.kneighbors(y)[0]
        y_nn = NearestNeighbors(n_neighbors=1, leaf_size=1, algorithm='kd_tree', metric=metric).fit(y)
        min_x_to_y = y_nn.kneighbors(x)[0]
        chamfer_dist = np.mean(min_y_to_x) + np.mean(min_x_to_y)
    else:
        raise ValueError("Invalid direction type. Supported types: \'y_x\', \'x_y\', \'bi\'")
        
    return chamfer_dist

In [28]:
import ast

from runner import test
import numpy as np

object_number = 1

#objects
Obj1_url = objects[object_number]["Obj1_url"]
Obj2_url = objects[object_number]["Obj2_url"]

#Before Evaluate do a Rotation and translation of
init_R_T = ((0.2, 0.2, 0.1), (-0.5, -0.5, -0.5))


#pipline name and pipline parameters
pipline_name = "general_pipeline"

#sampling
large_object = objects[object_number]["large_object"]
small_object = objects[object_number]["small_object"]


N = 15
Object_t1 = objects[object_number]["to1"]
Object_t2 = objects[object_number]["to2"]
Object_t3 = objects[object_number]["to3"]

border_t1 = objects[object_number]["tb1"]
border_t2 = objects[object_number]["tb1"]
border_t3 = objects[object_number]["tb1"]


#threshold for corner
thre = objects[object_number]["thre"]

dilation_size = objects[object_number]["dilation_size"]

pipline_variables = (small_object, large_object, N, Object_t1, Object_t2,
                     Object_t3, border_t1, border_t2, border_t3, dilation_size, thre)

#test
test_name = "ICP_test"

# eval 

eval_list = ["rms"]

test_breaking_thin = test(Obj2_url, Obj1_url, init_R_T, pipline_name, pipline_variables, test_name, eval_list, show_results = True, save_results=False)

my_results = test_breaking_thin.run()

_________________________First Object_________________________
start
piece3.ply
small object
my number is ->  0.0041171859741210935
starting init
starting calculating atts


100%|██████████████████████████████████████████████████████████████████████████| 30182/30182 [00:12<00:00, 2411.93it/s]


my radius is :  0.006752086811558033
347722
After graph 30182
constructing borders
3415
shortest_allowed_branch_length 584.3800133474792
smallest_isolated_island_length 584.3800133474792
shortest_cycle_length 584.3800133474792
my radius is :  0.012084650976664195
73975
After graph 3125
After Pruning 0
After Pruning 343
dilation and segmentation
borders dilated


174781it [05:05, 213.45it/s]                                                                                           

15155


...

2217


258509it [06:59, 182.25it/s]

10


258558it [07:01, 614.06it/s]


got faces
KNN Radius :  0.005674372796985649
35
expanded
coloring
_________________________Registeration_________________________
_________________________Evaluation_________________________


In [30]:
get_winner_pair(test_breaking_thin)

,index,o1,o2,chamfer
8,8,2,2,0.010871
10,10,3,1,0.026462
4,4,1,1,0.030961
7,7,2,1,0.042063
5,5,1,2,0.046042
11,11,3,2,0.046676
6,6,2,0,0.076597
3,3,1,0,0.088553
9,9,3,0,0.096831
1,1,0,1,0.103636


In [29]:
import pandas as pd
df_breaking_3 = pd.DataFrame(test_breaking_thin.results)
df_breaking_3["total"] = df_breaking_3["R_error"]+df_breaking_3["T_error"]
df_breaking_3.sort_values('T_error')

,o1,o2,R_error,T_error,total
8,2,2,5.445350,0.006108,5.451458
10,3,1,4.768334,0.084660,4.852994
3,1,0,5.147565,0.194037,5.341602
9,3,0,3.663123,0.299408,3.962531
7,2,1,5.871785,0.490908,6.362693
11,3,2,3.721386,0.544615,4.266001
5,1,2,4.914967,0.571343,5.486311
1,0,1,4.926005,0.638539,5.564544
6,2,0,4.686902,0.651526,5.338427
0,0,0,4.836084,0.651854,5.487938
